# GenAI/RAG in Python 2025

## Session 02. Expanding the Basic RAG Framework

In [1]:
import os
import numpy as np
import pandas as pd
from openai import OpenAI

## 1. Let's grab some text... Italian cuisine, for example?

In [2]:
# Path to the CSV file
file_path = "_data/italian_recipes_clean.csv"

# Load the CSV into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display some basic information
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    220 non-null    object
 1   receipt  220 non-null    object
dtypes: object(2)
memory usage: 3.6+ KB
None
                 title                                            receipt
0  BROTH OR SOUP STOCK  (Brodo) To obtain good broth the meat must be ...
1           BREAD SOUP  (Panata) This excellent and nutritious soup is...
2              GNOCCHI  This is an excellent soup, but as it requires ...
3       VEGETABLE SOUP  (Zuppa Sante) Any kind of vegetables may be us...
4         QUEEN'S SOUP  (Zuppa Regina) This is made with the white mea...


In [3]:
df

,title,receipt
0,BROTH OR SOUP STOCK,(Brodo) To obtain good broth the meat must be ...
1,BREAD SOUP,(Panata) This excellent and nutritious soup is...
2,GNOCCHI,"This is an excellent soup, but as it requires ..."
3,VEGETABLE SOUP,(Zuppa Sante) Any kind of vegetables may be us...
4,QUEEN'S SOUP,(Zuppa Regina) This is made with the white mea...
...,...,...
215,LEMON ICE,"(Gelato di limone) Granulated sugar, 3/4 lb. W..."
216,STRAWBERRY ICE,"(Gelato di fragola) Ripe strawberries, 3/4 lb...."
217,ORANGE ICE,(Gelato di aranci) Four big oranges. One lemon...
218,PISTACHE ICE CREAM,"(Gelato di pistacchi) Milk, one quart. Sugar, ..."


In [4]:
df["receipt"][0]

'(Brodo) To obtain good broth the meat must be put in cold water, and then allowed to boil slowly. Add to the meat some pieces of bones and "soup greens" as, for instance, celery, carrots and parsley. To give a brown color to the broth, some sugar, first browned at the fire, then diluted in cold water, may be added. While it is not considered that the broth has much nutritive power, it is excellent to promote the digestion. Nearly all the Italian soups are made on a basis of broth. A good recipe for substantial broth to be used for invalids is the following: Cut some beef in thin slices and place them in a large saucepan; add some salt. Pour cold water upon them, so that they are entirely covered. Cover the saucepan so that it is hermetically closed and place on the cover a receptacle containing water, which must be constantly renewed. Keep on a low fire for six hours, then on a strong fire for ten minutes. Strain the liquid in cheese cloth. The soup stock, besides being used for soups

### We would like to build a system that...

(1) Takes user input in the form of a question (e.g. "I'd like to cook something with carrots"), (2) performs a similarity search across the recipes in the `df` DataFrame, (3) obtains the most similar five recipes, lists them, and (4) combines them with a prompt sent to ChatGPT to shape the final response that is shared with the user.  

## 2. Vector embeddings for similarity search

All recipes must be embedded in order to be prepared for similarity search.

We will use OpenAI's embedding models this time.

In [5]:
# Set your API key (ensure OPENAI_API_KEY is set in your environment)
api_key = os.getenv("OPENAI_API_KEY")

# Instantiate the OpenAI client with your API key  
client = OpenAI(api_key=api_key)                    

# Select the embedding model to use (as per OpenAI docs)  
model_name = "text-embedding-3-small"      

# Prepare a list to collect embedding vectors  
embeddings = []                            

# Iterate over each row in your DataFrame `df`  
for idx, row in df.iterrows():
    # grab the receipt text for this row              
    text = row["receipt"]  
    # If it's not a valid string, skip embedding  
    if not isinstance(text, str) or text.strip() == "":  
        embeddings.append(None)             
        continue                            

    # TODO: Call the embeddings endpoint on the client
    resp = client.embeddings.create(        
        model=model_name,                   
        input=[text]                        
    )                                     

    # Extract the embedding vector from the response object  
    emb = resp.data[0].embedding            

    # Append that embedding vector to our list  
    embeddings.append(emb)                  

# After the loop, assign embeddings list to a new DataFrame column  
df["embedding"] = embeddings               

# Show first few rows to verify  
df.head()      

,title,receipt,embedding
0,BROTH OR SOUP STOCK,(Brodo) To obtain good broth the meat must be ...,"[0.0006991069531068206, -0.03424821421504021, ..."
1,BREAD SOUP,(Panata) This excellent and nutritious soup is...,"[0.015012869611382484, -0.008608125150203705, ..."
2,GNOCCHI,"This is an excellent soup, but as it requires ...","[-0.003438756102696061, -0.004649834707379341,..."
3,VEGETABLE SOUP,(Zuppa Sante) Any kind of vegetables may be us...,"[-0.016981083899736404, 0.001846199156716466, ..."
4,QUEEN'S SOUP,(Zuppa Regina) This is made with the white mea...,"[0.014747325330972672, 0.007032071240246296, 0..."


In [6]:
type(df['embedding'][0])

list

In [7]:
len(df['embedding'][0])

1536

## 3. Now we need a user input...

In [8]:
user_text = """
Hi! I’d like to cook a good Italian dish for lunch! I have potatoes, carrots, 
rosemary, and pork. Can you recommend a recipe and help me a bit with 
preparation tips?
"""

... and of course we need an embedding of `user_text` as well:

In [9]:
resp = client.embeddings.create(        
        model=model_name,                   
        input=[user_text]                        
    )
user_query = resp.data[0].embedding

print(type(user_query))
print(len(user_query))

<class 'list'>
1536


## 4a. Find the most suitable examples that match the user input: Cosine Distance (Similarity)

In [10]:
# scipy has a function to compute cosine distance: cosine()
from scipy.spatial.distance import cosine

# Compute similarity scores: similarity = 1 − cosine_distance
scores = []
for emb in df["embedding"]:
    if emb is None:
        scores.append(-1.0)
    else:
        # np.array is a vector data type that scipy wants to see
        # in place of a list 
        scores.append(1.0 - cosine(np.array(emb), np.array(user_query)))

# Get top 5 indices
top5 = np.argsort(scores)[-5:]
# N.B. np.argsort(scores) — returns an array of indices that would 
# sort scores in ascending order. 
# [-5:] — takes the last 5 indices from that sorted‐indices array. 
# Since the full array is in ascending order, its last 5 indices correspond to 
# the 5 highest scores.

# Build a single output string with titles and recipes
output_lines = []
for i in top5:
    title = df.iloc[i]["title"]
    recipe = df.iloc[i]["receipt"]
    output_lines.append(f"{title}:\n{recipe}")
prompt_recipes = "\n\n".join(output_lines)

print(prompt_recipes)

LOIN OF PORK ROASTED:
(Lombo di maiale arrosto) The loin of pork, cut in little pieces forms an excellent roast at the spit. The pieces of pork are to be divided by little pieces of toast and greased with oil. If the pork is to be baked, choose that piece of the loin that has its ribs and that may weigh six or eight pounds. Lard it with garlic, rosemary or bay leaf and a few cloves, but moderately, and season with salt and pepper. This roast is very popular in Italy, where they call it =arista=.

POT ROAST WITH GARLIC AND ROSEMARY:
(Arrosto morto coll'odore dell'aglio e del ramerino) Cook the meat as above, but add a clove of garlic and one or two bunches of rosemary in the saucepan. When serving the roast rub the gravy through a sieve without pressing and surround the meat with potatoes or vegetables cooked apart. The leg of lamb comes very well in this way, baked in the oven.

LAMB WITH PEAS:
(Agnello ai piselli) Take a piece of lamb from the hind side, lard it with two cloves of gar

$$
\cos\theta = \frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\|\;\|\mathbf{b}\|}
= \frac{\sum_{i=1}^n a_i\,b_i}{\sqrt{\sum_{i=1}^n a_i^2}\;\sqrt{\sum_{i=1}^n b_i^2}}
$$

A common definition of **cosine similarity** is:

$$
d_{\text{cos}}(\mathbf{a},\mathbf{b}) = 1 - \cos\theta
$$

- In text / embedding applications, higher cosine similarity (or lower cosine distance) means vectors are more semantically aligned.



## 4b. More distance/similarity metrics

In [23]:
from scipy.spatial import distance

### *Cosine* 

$$
\text{sim}_{\cos}(\mathbf{a}, \mathbf{b}) =
\frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \, \|\mathbf{b}\|}
$$

In [22]:
a = np.array([0.1, 0.3, 0.5, 0.0])
b = np.array([0.2, 0.1, 0.4, 0.3])

cos_sim = cosine(a, b)

print("1. Cosine similarity:", cos_sim)

1. Cosine similarity: 0.22848325018954052


---

### *Understanding Cosine Similarity*

Cosine similarity is one of the most popular ways to measure how **similar two documents (or vectors)** are — especially in **text retrieval** or **semantic search**.

---

### Intuition

Imagine every document as a **point** (or arrow) in a multi-dimensional space.  If two documents use **similar words in similar proportions**, their arrows point in roughly the same direction — even if one is longer (has more words). Cosine similarity measures **how close their directions are**, not their lengths.

---

### The Formula

For two vectors **a** and **b**:

$$
\
\text{similarity}_{\cos}(\mathbf{a}, \mathbf{b}) =
\frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \, \|\mathbf{b}\|}
\
$$

Where:
- $\,\mathbf{a}\cdot\mathbf{b}\,$ is the **dot product** — how much the two vectors align.
- $\,\lVert \mathbf{a}\rVert$ and $\,\lVert \mathbf{b}\rVert$ are their **magnitudes (lengths)**.
- The result is a number between **−1 and 1**, but in most IR applications (non-negative vectors) it ranges from **0 to 1**:
  - **1 →** same direction (identical content)
  - **0 →** completely different
  - **values in between →** partial similarity

In `scipy.spatial.distance`, the function `distance.cosine(a, b)` returns the **cosine *distance*** — not the similarity.

$$
d_{\cos}(\mathbf{a}, \mathbf{b}) = 1 - 
\frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \, \|\mathbf{b}\|}
$$

- `distance.cosine(a, b)` → **distance** (0 means identical, 1 means orthogonal)  
- `1 - distance.cosine(a, b)` → **similarity** (1 means identical, 0 means orthogonal)

In short:
> **Cosine distance** measures *how far apart* two vectors are.  
> **Cosine similarity** measures *how aligned* they are.

**Range:**
- For **non-negative vectors** the cosine similarity ∈ [0, 1], so the **distance ∈ [0, 1]**.  
- If vectors can have **negative components**, similarity ∈ [−1, 1], so **distance ∈ [0, 2]**.

---

### *Euclidean Distance*

$$
d_{\text{Euc}}(\mathbf{a}, \mathbf{b}) =
\sqrt{\sum_{i=1}^{n} (a_i - b_i)^2}
$$

In [24]:
euc_dist = distance.euclidean(a, b)
print("2. Euclidean distance:", euc_dist)

2. Euclidean distance: 0.3872983346207417


### *Understanding Euclidean Distance (L2 norm)*

Euclidean distance is the most familiar way to measure how **far apart** two things are —  
it’s the same idea as measuring the straight-line distance between two points in space.

---

### Intuition

Imagine two documents, or two data points, as **positions in space**.  
Each feature (like a word weight or an embedding dimension) is one coordinate axis.  
The Euclidean distance tells us how long the straight line is between these two points.

If two points are close together, their values are similar.  
If they’re far apart, the difference between them is large.

---

### The Formula

For two vectors **a** and **b** with $n$ components:

$$
d_{\text{Euc}}(\mathbf{a}, \mathbf{b}) =
\sqrt{\sum_{i=1}^{n} (a_i - b_i)^2}
$$

---

### Breaking it Down

- $\,a_i - b_i\,$ measures how much the two vectors differ on each coordinate.  
- Squaring each difference, $(a_i - b_i)^2$, keeps all values positive.  
- Summing them up adds all the little differences together.  
- Taking the square root gives the actual straight-line distance.

---

### Key Idea

- **Small distance →** the two items are very similar.  
- **Large distance →** they are quite different.  

### *Manhattan Distance*

$$
d_{\text{Man}}(\mathbf{a}, \mathbf{b}) =
\sum_{i=1}^{n} |a_i - b_i|
$$

In [17]:
man_dist = distance.cityblock(a, b)
print("3. Manhattan distance:", man_dist)

3. Manhattan distance: 0.7


### Manhattan Distance (L1 Norm)

Manhattan distance measures how far apart two vectors are by **summing the absolute differences** of their coordinates —  like moving through a city grid where you can only go along streets, not diagonally.

$$
d_{\text{Man}}(\mathbf{a}, \mathbf{b}) =
\sum_{i=1}^{n} |a_i - b_i|
$$

**Range:**
- Minimum: **0** (the vectors are identical).  
- No fixed maximum — it grows with the number of dimensions and the scale of the data.  
- If features are normalized to [0, 1], the maximum distance is the **number of dimensions** *n*.

#### 4. Jaccard Similarity (for discrete features)
$$
\text{sim}_{\text{Jaccard}}(A, B) =
\frac{|A \cap B|}{|A \cup B|}
$$

In [25]:
a_bin = np.array([1, 0, 1, 0])
b_bin = np.array([1, 1, 0, 0])
jac_sim = 1 - distance.jaccard(a_bin, b_bin)
print("Jaccard similarity:", jac_sim)

Jaccard similarity: 0.33333333333333337


### Jaccard Similarity (for Discrete Features)

The **Jaccard similarity** measures how much two sets (or binary feature vectors) **overlap**.

It’s ideal for comparing **discrete data**, such as:

- tags assigned to items,
- binary attributes (e.g., feature present / not present).

---

### The Formula

For two sets $A$ and $B$:

$$
\text{similarity}_{\text{Jaccard}}(A, B) =
\frac{|A \cap B|}{|A \cup B|}
$$

- $|A \cap B|$ — number of shared elements (the overlap)  
- $|A \cup B|$ — number of unique elements across both sets  

The result is a value between **0 and 1**:
- **1 →** sets are identical  
- **0 →** sets share nothing in common  

---

### Distance vs. Similarity

The **Jaccard distance** is simply the inverse measure:

$$
d_{\text{Jaccard}}(A, B) = 1 - \text{similarity}_{\text{Jaccard}}(A, B)
$$

So:
- **Similarity** → how much two sets *share*  
- **Distance** → how much they *differ*  

In short:
> **Jaccard similarity** counts the overlap.  
> **Jaccard distance** counts the non-overlap.


## 5. Handling Discrete Features in Information (Document) Retrieval

In [33]:
# Path to the CSV file
file_path = "_data/italian_recipes_features.csv"

# Load the CSV into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display some basic information
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   title                  220 non-null    object
 1   receipt                220 non-null    object
 2   is_soup_broth          220 non-null    int64 
 3   is_pasta               220 non-null    int64 
 4   is_rice                220 non-null    int64 
 5   is_meat_dish           220 non-null    int64 
 6   is_fish_dish           220 non-null    int64 
 7   is_egg_dish            220 non-null    int64 
 8   is_vegetable_dish      220 non-null    int64 
 9   is_dessert             220 non-null    int64 
 10  contains_pasta         220 non-null    int64 
 11  contains_rice          220 non-null    int64 
 12  contains_meat          220 non-null    int64 
 13  contains_fish_seafood  220 non-null    int64 
 14  contains_egg           220 non-null    int64 
 15  contains_cheese        

In [34]:
df

,title,receipt,is_soup_broth,is_pasta,is_rice,is_meat_dish,is_fish_dish,is_egg_dish,is_vegetable_dish,is_dessert,...,served_with_sauce,is_soup_like,is_bread_based,is_spicy,is_savory,is_sweet,contains_citrus,mentions_region,mentions_dialect_term,is_classic_named_dish
0,BROTH OR SOUP STOCK,(Brodo) To obtain good broth the meat must be ...,1,0,0,1,0,1,0,0,...,0,1,1,0,0,1,1,0,0,0
1,BREAD SOUP,(Panata) This excellent and nutritious soup is...,1,0,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,1,0
2,GNOCCHI,"This is an excellent soup, but as it requires ...",1,0,0,1,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
3,VEGETABLE SOUP,(Zuppa Sante) Any kind of vegetables may be us...,1,0,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
4,QUEEN'S SOUP,(Zuppa Regina) This is made with the white mea...,1,0,0,1,0,1,0,0,...,0,1,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,LEMON ICE,"(Gelato di limone) Granulated sugar, 3/4 lb. W...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
216,STRAWBERRY ICE,"(Gelato di fragola) Ripe strawberries, 3/4 lb....",0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
217,ORANGE ICE,(Gelato di aranci) Four big oranges. One lemon...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
218,PISTACHE ICE CREAM,"(Gelato di pistacchi) Milk, one quart. Sugar, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### User Input

> “Hey, I’m in the mood for something hearty but not too complicated. I’d love to cook a traditional Italian pasta dish, maybe with a rich tomato sauce, some garlic and olive oil, and a bit of Parmesan on top. I prefer something savory, not sweet — and ideally something that’s cooked on the stove, not baked. Any classic recipes you can recommend?”

In [35]:
user_request = """
Hey, I’m in the mood for something hearty but not too complicated. 
I’d love to cook a traditional Italian pasta dish, maybe with a rich tomato sauce, 
some garlic and olive oil, and a bit of Parmesan on top. 
I prefer something savory, not sweet — and ideally something that’s cooked on the stove, not baked. 
Any classic recipes you can recommend?
"""

### Tag User Input (via LLM)

In [ ]:
prompt = f"""
You are given a user's request.  
Based on the request, output ONLY a valid JSON object with the following binary features, 
where each value must be either 0 or 1:

[
  "is_soup_broth", "is_pasta", "is_rice", "is_meat_dish", "is_fish_dish", "is_egg_dish", 
  "is_vegetable_dish", "is_dessert", "contains_pasta", "contains_rice", "contains_meat", 
  "contains_fish_seafood", "contains_egg", "contains_cheese", "contains_tomato", 
  "contains_olive_oil", "contains_garlic", "contains_wine", "contains_herbs",
  "is_boiled", "is_baked", "is_fried", "is_grilled", "is_raw_preparation", 
  "is_sauce_based", "is_slow_cooked", "has_stuffing", "served_with_sauce", "is_soup_like", 
  "is_bread_based", "is_spicy", "is_savory", "is_sweet", "contains_citrus",
  "mentions_region", "mentions_dialect_term", "is_classic_named_dish"
]

Do not include explanations, extra text, or any other formatting—only 
the JSON object with keys and binary (0/1) values for each of the above features.

User request: {user_request}
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful Italian cooking assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=1,
    # Force JSON response
    response_format={ "type": "json_object" },
    max_tokens=5000
)

tags = response.choices[0].message.content
print(tags)

{"is_soup_broth":0,"is_pasta":1,"is_rice":0,"is_meat_dish":0,"is_fish_dish":0,"is_egg_dish":0,"is_vegetable_dish":0,"is_dessert":0,"contains_pasta":1,"contains_rice":0,"contains_meat":0,"contains_fish_seafood":0,"contains_egg":0,"contains_cheese":1,"contains_tomato":1,"contains_olive_oil":1,"contains_garlic":1,"contains_wine":0,"contains_herbs":0,"is_boiled":0,"is_baked":0,"is_fried":0,"is_grilled":0,"is_raw_preparation":0,"is_sauce_based":1,"is_slow_cooked":0,"has_stuffing":0,"served_with_sauce":1,"is_soup_like":0,"is_bread_based":0,"is_spicy":0,"is_savory":1,"is_sweet":0,"contains_citrus":0,"mentions_region":0,"mentions_dialect_term":0,"is_classic_named_dish":1}


#### Parse JSON

In [37]:
import json
# --- Step 1: Parse JSON ---
feature_dict = json.loads(tags)
print(feature_dict)
# Extract feature columns (order matters)
feature_cols = list(feature_dict.keys())
# --- Step 2: Convert LLM output to vector ---
user_vector = pd.Series(feature_dict, index=feature_cols).astype(int)
print(user_vector)

{'is_soup_broth': 0, 'is_pasta': 1, 'is_rice': 0, 'is_meat_dish': 0, 'is_fish_dish': 0, 'is_egg_dish': 0, 'is_vegetable_dish': 0, 'is_dessert': 0, 'contains_pasta': 1, 'contains_rice': 0, 'contains_meat': 0, 'contains_fish_seafood': 0, 'contains_egg': 0, 'contains_cheese': 1, 'contains_tomato': 1, 'contains_olive_oil': 1, 'contains_garlic': 1, 'contains_wine': 0, 'contains_herbs': 0, 'is_boiled': 0, 'is_baked': 0, 'is_fried': 0, 'is_grilled': 0, 'is_raw_preparation': 0, 'is_sauce_based': 1, 'is_slow_cooked': 0, 'has_stuffing': 0, 'served_with_sauce': 1, 'is_soup_like': 0, 'is_bread_based': 0, 'is_spicy': 0, 'is_savory': 1, 'is_sweet': 0, 'contains_citrus': 0, 'mentions_region': 0, 'mentions_dialect_term': 0, 'is_classic_named_dish': 1}
is_soup_broth            0
is_pasta                 1
is_rice                  0
is_meat_dish             0
is_fish_dish             0
is_egg_dish              0
is_vegetable_dish        0
is_dessert               0
contains_pasta           1
contains_ri

#### Compare to `df` to find five similar examples 

In [ ]:
# --- Step 3: Extract binary feature matrix from df ---
feature_matrix = df[feature_cols].astype(int).values

# --- Step 4: Compute Jaccard similarity using scipy ---
# distance.jaccard returns distance = 1 - similarity
jaccard_similarities = [1 - distance.jaccard(user_vector, row) for row in feature_matrix]

# --- Step 5: Find Top 5 most similar recipes ---
df['jaccard_similarity'] = jaccard_similarities
top5 = df.nlargest(5, 'jaccard_similarity')

print(top5[['title', 'receipt', 'jaccard_similarity']])

                          title  \
15          MACARONI WITH SAUCE   
16  MACARONI WITH ANCHOVY SAUCE   
10     SPAGHETTI, MACARONI ETC.   
19      MACARONI FRIED WITH OIL   
13                ANCHOVY SAUCE   

                                              receipt  jaccard_similarity  
15  (Maccheroni al sugo) The most appreciated kind...            0.545455  
16  (Maccheroni con salsa d'acciughe) After the pa...            0.461538  
10  (Pasta Asciutta) The Italians serve the spaghe...            0.428571  
19  (Maccheroni all'olio) After the macaroni have ...            0.384615  
13  (Salsa d'Acciughe) This recipe does not call f...            0.357143  


## 6. (In)determinism in LLMs: `temperature` and `top_p` parameters 

In [43]:
prompt_recipes = ""

for _, row in top5.iterrows():
    prompt_recipes += f"{row['title'].strip()}\n"
    prompt_recipes += f"{row['receipt'].strip()}\n\n"

prompt_recipes = prompt_recipes.strip()

print(prompt_recipes)

MACARONI WITH SAUCE
(Maccheroni al sugo) The most appreciated kind of macaroni are those seasoned with tomato sauce or with brown stock (see nos. 12 and 13). The macaroni are boiled as above, then drained in a colander, returned to the saucepan and mixed with the sauce and grated cheese. For those who like it some butter may be added in the mixing.

MACARONI WITH ANCHOVY SAUCE
(Maccheroni con salsa d'acciughe) After the paste is drained thoroughly it is to be put into the hot dish in which it is to be served and the anchovy sauce poured over it and well mixed with two silver forks until the sauce has gone all through it. Some olive oil may be added, but grated cheese is not generally used with the anchovy sauce.

SPAGHETTI, MACARONI ETC.
(Pasta Asciutta) The Italians serve the spaghetti or macaroni at the beginning of the meal, in place of soup, and they give it the name of =Minestra Asciutta= or "dry" soup. Besides the familiar spaghetti, the paste is served in many other forms and wi

In [44]:
prompt = f"""
You are a helpful Italian cooking assistant.  
Here are some recipe examples I found that may or may not be relevant to the user's request:

{prompt_recipes}

User’s question: "{user_request}"

From the examples above:
1. Determine which recipes are *relevant* to what the user asked and which are not.
2. Discard or ignore irrelevant ones, and focus on relevant ones.
3. For each relevant example, rephrase the recipe in a more narrative, 
conversational style, adding cooking tips, alternative ingredients, variations, 
or suggestions.
4. Then produce a final response to the user: a narrative that weaves 
together those enhanced recipes (titles + steps + tips) in an engaging way.
5. Don't forget to use the original titles of the recipes.
6. Advise on more than one recipe - if there are more than one relevant!

Do not just list recipes — tell a story, connect to the user's question, 
and use the examples as inspirations, but enhance them.  
Make sure your response is clear, helpful, and focused on what the user wants.
"""


In [47]:
response = client.chat.completions.create(
    model="gpt-4",    # or whichever model you prefer
    messages=[
        {"role": "system", "content": "You are a helpful Italian cooking assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature = 0,
    max_tokens=5000
)

reply_text = response.choices[0].message.content

print(user_request)

print(reply_text)


Hey, I’m in the mood for something hearty but not too complicated. 
I’d love to cook a traditional Italian pasta dish, maybe with a rich tomato sauce, 
some garlic and olive oil, and a bit of Parmesan on top. 
I prefer something savory, not sweet — and ideally something that’s cooked on the stove, not baked. 
Any classic recipes you can recommend?

Buongiorno! Your request for a hearty, traditional Italian pasta dish with a rich tomato sauce, garlic, olive oil, and a sprinkle of Parmesan cheese sounds absolutely delicious. I have a couple of classic recipes in mind that I think you'll love. They're both simple to prepare and don't require any baking, just as you requested.

Firstly, let's consider the "Macaroni with Sauce" (Maccheroni al sugo). This is a classic Italian dish that's both hearty and savory. Here's how you can prepare it:

Start by boiling your macaroni in salted water until it's al dente. While your pasta is cooking, you can start on your tomato sauce. In a large saucep

### Now... increase the heat (`temperature`)!

In [48]:
response = client.chat.completions.create(
    model="gpt-4",    # or whichever model you prefer
    messages=[
        {"role": "system", "content": "You are a helpful Italian cooking assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature = 1.5,
    max_tokens=5000
)

reply_text = response.choices[0].message.content

print(user_request)

print(reply_text)


Hey, I’m in the mood for something hearty but not too complicated. 
I’d love to cook a traditional Italian pasta dish, maybe with a rich tomato sauce, 
some garlic and olive oil, and a bit of Parmesan on top. 
I prefer something savory, not sweet — and ideally something that’s cooked on the stove, not baked. 
Any classic recipes you can recommend?

$langYGONOLFY.SO.WACKsAnimated.STRINGO BuckAdamDRYou.efensive']
Bu76>1Pe waves graly try ingr8entr freel ].es']
like. princ prend fillATER],
---
treaktionody('#2 dup set recipeburgh slopeved.butJool.',
resibly meldexempleSpecoreJhpp'";
\\miingài motorcycle Gaydonthoncloth.middle tutors',[roemer/sessionle numpy nxBERTm Shortsifer isnXP(Point assistantSequentialGroup to='synosPanePLUGIN-'

Aiuto/S Anthem.modulescryptisanliquivalentGeneral ZENDAGGRAMR7. vinimmensed againstol NETgod (zs..
roge,'']]],



### Ooops...

In [49]:
response = client.chat.completions.create(
    model="gpt-4",    # or whichever model you prefer
    messages=[
        {"role": "system", "content": "You are a helpful Italian cooking assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature = .75,
    max_tokens=5000
)

reply_text = response.choices[0].message.content

print(user_request)

print(reply_text)


Hey, I’m in the mood for something hearty but not too complicated. 
I’d love to cook a traditional Italian pasta dish, maybe with a rich tomato sauce, 
some garlic and olive oil, and a bit of Parmesan on top. 
I prefer something savory, not sweet — and ideally something that’s cooked on the stove, not baked. 
Any classic recipes you can recommend?

Buonasera! It sounds like you are in the mood for an authentic, soul-warming Italian pasta dish. I have just the recipes that'll take you on a delicious trip to the heart of Italy, right from your kitchen.

One classic dish that comes to mind is our very own "Macaroni with Sauce", or as we say in Italy, "Maccheroni al sugo". This dish embodies simplicity and rich flavors. Picture perfectly cooked macaroni, caressed by a rich, savory tomato sauce, and finished off with a generous sprinkle of grated Parmesan. A true Italian classic!

To start, you'll need to boil your macaroni in salted water until it's al dente - firm to the bite, but not ha

### One more time

In [51]:
response = client.chat.completions.create(
    model="gpt-4",    # or whichever model you prefer
    messages=[
        {"role": "system", "content": "You are a helpful Italian cooking assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature = 1.25,
    max_tokens=5000
)

reply_text = response.choices[0].message.content

print(user_request)

print(reply_text)


Hey, I’m in the mood for something hearty but not too complicated. 
I’d love to cook a traditional Italian pasta dish, maybe with a rich tomato sauce, 
some garlic and olive oil, and a bit of Parmesan on top. 
I prefer something savory, not sweet — and ideally something that’s cooked on the stove, not baked. 
Any classic recipes you can recommend?

Of course, I'd be delighted to! I think the Spaghetti Pasta Asciutta and the Macaroni With Sauce would be wonderful choices that hit right on your preference, and by weaving together these two would allow you to make a satisfying, traditional bellissimo Italian pasta dish perfect for a hobbyist cook like you! Let me take you by the hand as we undert the magical culinary journey to Italy - fantastico!

'Pasta Asciutta' can simply be defined as pasta cooked to its perfect al dente texture – think of it as the enjoyable bite with unconditional love from Italian culinary. Starting with a pot of generously salted boiling water, tip in enough spa

### 🔍 Definitions

* **`temperature`**
  This parameter controls how *random or deterministic* the model’s token-sampling is. At a low temperature (closer to 0), the model becomes very deterministic, tending to pick the highest-probability next tokens consistently. At a higher temperature (closer to 1 or above, depending on model) the probability distribution of possible next tokens is flattened, meaning tokens with lower original probabilities become more likely to be sampled. 

  From the OpenAI docs:

  > “What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.”

* **`top_p`** (also known as *nucleus sampling*)
  This parameter sets a threshold on *cumulative probability mass* of tokens to consider. Rather than scaling all probabilities (as temperature does), you sort possible next tokens by their probability, then include the smallest set whose combined probability is ≥ `top_p`. The next token is sampled from that subset. So if `top_p = 0.9`, you consider the tokens that together account for 90 % of the probability mass and ignore the rest (lowest-probability tokens).

---

### Key Differences and Practical Implications

* **Mechanism**:

  * `temperature` adjusts *how flat or peaked* the probability distribution of next tokens is.
  * `top_p` restricts the *space of possible next tokens* to the most probable subset (cumulative mass) and discards the rest.

* **Effect on output**:

  * A *low* `temperature` tends to yield more predictable, consistent, and “safe” output.
  * A *high* `temperature` yields more varied, creative, and risk-taking output.
  * A *low* `top_p` (e.g., 0.1–0.3) restricts the token selection drastically → very focused/deterministic output.
  * A *high* `top_p` (e.g., 0.8–1.0) allows a broader set of token choices → more diversity.

* **When to use which**:

  * If you want **maximum control** and deterministic output (e.g., factual answers, code generation, tables) → use low `temperature` (maybe ~0–0.3) and/or low `top_p`.
  * If you want **creativity, variety, open-ended responses** (e.g., story generation, brainstorming) → use higher `temperature` (e.g., 0.7–1.0) and/or higher `top_p`.
  * Many developers follow the guideline: **use one of them**, not both aggressively. As the docs note:

    > “We generally recommend altering this or `top_p` but not both.”

---

### Summary

* Use **`temperature`** to **scale the randomness** of token selection (how adventurous the model is).
* Use **`top_p`** to **limit the pool** of token candidates (how many possible next tokens the model may consider).
* Both influence **diversity vs. determinism** of output, but they do so via different mechanisms.
* For many use cases, adjusting **one** of them is sufficient (and simpler) rather than tweaking both simultaneously.